In [58]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from kaggle.api.kaggle_api_extended import KaggleApi #sometimes throws errors and has to be rerun for some reason

In [59]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.clear_session()

In [60]:
api = KaggleApi()
folder_path = 'Dataset'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    api.dataset_download_files('wanderdust/coin-images', path=folder_path, unzip=True)
else:
    print('Dataset already downloaded')

Dataset already downloaded


In [61]:
num_classes = 211
input_size = 164

In [62]:
with open('Dataset/cat_to_name.json','r') as f:
    coin_data = json.load(f)

In [63]:
image_paths = {'train':[],'validation':[],'test':[]}
labels = {'train':[],'validation':[],'test':[]}
paths = {'train':'Dataset/coins/data/train',
         'validation':'Dataset/coins/data/validation/',
         'test': 'Dataset/coins/data/test/'
         }
for k in ['train', 'validation', 'test']:
    for parent_file_name in os.listdir(paths[k]):
        for file_name in os.listdir(os.path.join(paths[k],parent_file_name)):
            if file_name.endswith(('.jpg','.png','.jpeg')):
                image_paths[k].append(os.path.join(paths[k],parent_file_name,file_name))
                labels[k].append(parent_file_name)

In [64]:
def image_preprocessing(image_path, label,num_classes,augment = False):
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32) 
    image = tf.image.resize(image, [input_size, input_size])
    label = tf.one_hot(int(label),num_classes)
    
    if augment:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
        image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
          
    return image, label  

dataset = {}


for k in image_paths:

    image_list = image_paths[k]
    label_list = labels[k]
    
    

    tf_dataset = tf.data.Dataset.from_tensor_slices((image_list, label_list))
    
    tf_dataset = tf_dataset.map(lambda image_path, label: image_preprocessing(image_path, label, num_classes), 
                                num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    if k == 'train':  
        
        tf_augmented_dataset = tf.data.Dataset.from_tensor_slices((image_list, label_list))
        tf_augmented_dataset = tf_augmented_dataset.map(
            lambda image_path, label: image_preprocessing(image_path, label, num_classes, augment=True),
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        tf_dataset = tf_dataset.concatenate(tf_augmented_dataset)

    tf_dataset = tf_dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
    
    dataset[k] = tf_dataset


In [65]:
def residual_block(x, filters, strides=1, kernel_size=(3,3)):
  
    shortcut = x

    x = Conv2D(filters, kernel_size=(kernel_size), strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=-1)(x)
    x = LeakyReLU()(x)

    x = Conv2D(filters, kernel_size=(kernel_size), strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=-1)(x)

    if x.shape[-1] != shortcut.shape[-1]:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same', use_bias=False)(shortcut)
        shortcut = BatchNormalization(axis=-1)(shortcut)

    x = Add()([x, shortcut])

    x = LeakyReLU()(x)

    return x

def build_resnet50(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same', use_bias=False)(input_layer)
    x = BatchNormalization(axis=-1)(x)
    x = LeakyReLU()(x)
    
    x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    
    x = Dropout(0.2)(x)


    for _ in range(3):
        x = residual_block(x, 64)


    x = residual_block(x, 128, strides=2)  
    for _ in range(3):
        x = residual_block(x, 128)


    x = residual_block(x, 256, strides=2)  
    for _ in range(5):
        x = residual_block(x, 256)

  
    x = residual_block(x, 512, strides=2)  
    for _ in range(2):
        x = residual_block(x, 512)

  
    x = GlobalAveragePooling2D()(x)
    
    x = Dropout(0.2)(x)


    x = Flatten()(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

model = build_resnet50((input_size,input_size,3), num_classes)
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

model.fit(dataset['train'], validation_data=(dataset['validation']),batch_size=32,epochs=64)

Epoch 1/64
401/401 [==============================] - 83s 196ms/step - loss: 4.5637 - accuracy: 0.0996 - val_loss: 5.5369 - val_accuracy: 0.0450
Epoch 2/64
401/401 [==============================] - 70s 169ms/step - loss: 3.5330 - accuracy: 0.2478 - val_loss: 4.2910 - val_accuracy: 0.1896
Epoch 3/64
401/401 [==============================] - 70s 169ms/step - loss: 2.6844 - accuracy: 0.4082 - val_loss: 4.0905 - val_accuracy: 0.2287
Epoch 4/64
401/401 [==============================] - 70s 170ms/step - loss: 2.0170 - accuracy: 0.5437 - val_loss: 3.0901 - val_accuracy: 0.3578
Epoch 5/64
401/401 [==============================] - 70s 169ms/step - loss: 1.5581 - accuracy: 0.6401 - val_loss: 2.4658 - val_accuracy: 0.4775
Epoch 6/64
401/401 [==============================] - 70s 170ms/step - loss: 1.2081 - accuracy: 0.7168 - val_loss: 2.1080 - val_accuracy: 0.5344
Epoch 7/64
401/401 [==============================] - 71s 172ms/step - loss: 0.9386 - accuracy: 0.7768 - val_loss: 2.2346 - val_ac

In [67]:
loss, accuracy = model.evaluate(dataset['test'])
print(f'Testing results:\n Loss: {loss}\n Accuracy: {accuracy}')
model.save('coin_man')

27/27 [==============================] - 3s 50ms/step - loss: 1.1030 - accuracy: 0.7986
Testing results:
 Loss: 1.1029906272888184
 Accuracy: 0.7985782027244568


INFO:tensorflow:Assets written to: coin_man\assets


INFO:tensorflow:Assets written to: coin_man\assets
